# 类与对象

## 改变对象的字符串显示
问题：想改变对象实例的打印内容，增加可读性
解决：重新定义 __ str __ () 和 __ repr __ () 方法

- 使用 repr() 函数返回 __ repr __ () 方法定义的字符串
- 使用 str() 或 print() 函数会返回 __ str __ () 方法定义的字符串

In [5]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'MyPair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(3, 4)
print(repr(p))
print(p)


MyPair(3, 4)
(3, 4)
